In [1]:
from urllib.parse import urlencode
import requests

In [2]:
base_url = 'https://api.github.com/search/repositories?q='

#Construct query
query = {}
query['pushed'] = '2021-01-01' #repositories pushed at date
query['sort'] = 'stars-desc' #sort by number of stars in descending order
query_str = ''
for key in query:
    query_str += str(key) + ':' + query[key]
    query_str += '+'
    
#Construct params
params = {}
params['per_page'] = 100 #number of results per page
params['page'] = 2 #return result in page #

#Query URL
url = base_url + query_str + '&' + urlencode(params)

print(url)


https://api.github.com/search/repositories?q=pushed:2021-01-01+sort:stars-desc+&per_page=100&page=2


In [12]:
user = 'user' #fill in username
token = 'token' #fill in token
response = requests.get(url, auth = requests.auth.HTTPBasicAuth(user, token))
response.headers

{'Server': 'GitHub.com', 'Date': 'Wed, 18 May 2022 02:23:03 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Cache-Control': 'no-cache', 'Vary': 'Accept, Authorization, Cookie, X-GitHub-OTP, Accept-Encoding, Accept, X-Requested-With', 'X-OAuth-Scopes': 'admin:enterprise, admin:gpg_key, admin:org, admin:org_hook, admin:public_key, admin:repo_hook, delete:packages, delete_repo, gist, notifications, repo, user, workflow, write:discussion, write:packages', 'X-Accepted-OAuth-Scopes': '', 'github-authentication-token-expiration': '2022-06-16 20:53:42 UTC', 'X-GitHub-Media-Type': 'github.v3; format=json', 'Link': '<https://api.github.com/search/repositories?q=pushed%3A2021-01-01+sort%3Astars-desc+&per_page=100&page=1>; rel="prev", <https://api.github.com/search/repositories?q=pushed%3A2021-01-01+sort%3Astars-desc+&per_page=100&page=3>; rel="next", <https://api.github.com/search/repositories?q=pushed%3A2021-01-01+sort%3Astars-desc+&per_page=100&page=10>

In [10]:
data = response.json()
data

{'total_count': 38743,
 'incomplete_results': False,
 'items': [{'id': 127434800,
   'node_id': 'MDEwOlJlcG9zaXRvcnkxMjc0MzQ4MDA=',
   'name': 'callbag-delay',
   'full_name': 'janryWang/callbag-delay',
   'private': False,
   'owner': {'login': 'janryWang',
    'id': 4060976,
    'node_id': 'MDQ6VXNlcjQwNjA5NzY=',
    'avatar_url': 'https://avatars.githubusercontent.com/u/4060976?v=4',
    'gravatar_id': '',
    'url': 'https://api.github.com/users/janryWang',
    'html_url': 'https://github.com/janryWang',
    'followers_url': 'https://api.github.com/users/janryWang/followers',
    'following_url': 'https://api.github.com/users/janryWang/following{/other_user}',
    'gists_url': 'https://api.github.com/users/janryWang/gists{/gist_id}',
    'starred_url': 'https://api.github.com/users/janryWang/starred{/owner}{/repo}',
    'subscriptions_url': 'https://api.github.com/users/janryWang/subscriptions',
    'organizations_url': 'https://api.github.com/users/janryWang/orgs',
    'repos_url'